In [4]:
import math
import numpy as np
import pandas as pd
import requests
import io

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Input Test A and B sensitivities and specificities and disease prevalance
## input prevalance as a comma separated list of possibles to be evaluated

suggestions: layton = 0.00052, ut % positive tests = 0.046, salt lake high = .00415

In [5]:
#characteristics of the tests
#Test A is Serology-IGM antibodies Sens = 83.6-96.6 Spec = 97.7-99.8
#Test B PCR 
sensA = .836
specA = .977
sensB = .63
specB = .99
sero_sens = .836
sero_spec = .977
pcr_sens = .63
pcr_spec = .99
vital_sens = .4
vital_spec = .67

test_list = [sero_sens,sero_spec,pcr_sens,pcr_spec,vital_sens,vital_spec]
#disease prevalance in the population
### Layton is at .052%, positive tests in UT = 4.5%, salt lake high = .415%
# for example prev = (.050,.010,.1)
prev_list = (0.045,.1,.2)

pop=350

### for reference, here are the sensitivies and specificities used earlier
#pcr option 1
sen_pcr1 = .63
sen_pcr2 = .73
spec_pcr = .99
#serology
sen_ser = .841
spec_ser = .923

#pick which test to evaluate/combine
#0,1 = se,sp serology
#2,3 = se,sp pcr
#4,5 = se,sp vitals
#ex check pcr twice test_pick = [0,1,0,1]
test_pick=[0,1,4,5]

In [6]:
S1 = test_list[test_pick[0]]+test_list[test_pick[2]]-(test_list[test_pick[0]]*test_list[test_pick[2]])
SP1 = test_list[test_pick[1]]*test_list[test_pick[3]]

In [7]:
test_list[test_pick[3]]

0.67

In [9]:
pop=1000
pre = .2
tp = pop*pre*.4
fn = (pop*pre)-(pop*pre*.4)
fp = pop*pre*.4
tn = pop-(pop*pre*.4+((pop*pre)-(pop*pre*.4))+pop*pre*.4)
print(tp,fn,fp,tn)
sens = tp/(tp+fn)
spec = tn/(fp+tn)
print(pre,sens,spec)

80.0 120.0 80.0 720.0
0.2 0.4 0.9


In [10]:
vital_List = []
pop=1000

prev_list=[.045,.1,.2]
for prev in prev_list:
    fn = []
    tp = []
    fp = []
    tn = []
    pop = 1000
    tp = pop*prev*.4
    fn = (pop*prev)-tp
    fp = pop*prev*.4
    tn = pop-(fp+tp+fn)
    
    print (prev,pop,tp,fp,tn,fn)
    sens = tp/(tp+fn)
    spec = tn/(fp+tn)
    print(prev,sens,spec)

0.045 1000 18.0 18.0 937.0 27.0
0.045 0.4 0.981151832460733
0.1 1000 40.0 40.0 860.0 60.0
0.1 0.4 0.9555555555555556
0.2 1000 80.0 80.0 720.0 120.0
0.2 0.4 0.9


In [11]:
fp+tp+fn

280.0

In [12]:
#FINAL CELL

Test_Char = pd.DataFrame(columns =('Prevalence','Test Name','Sensitivity','Specificitiy','NegPredVal','Pop_Size','Infected','False_Negatives'))
dt_df = pd.DataFrame(columns =('Prevalence','Test Name','Sensitivity','Specificitiy','NegPredVal','Pop_Size','Infected','False_Negatives'))

for prev in prev_list:
    for i in [[0,1,0,1],[2,3,2,3],[0,1,2,3],[2,3,4,5],[0,1,4,5]]:
        
        #0,1 = se,sp serology
        #2,3 = se,sp pcr
        #4,5 = se,sp vitals
        #ex check pcr twice test_pick = [0,1,0,1]
        test_pick = i
        POsens = test_list[test_pick[0]]+test_list[test_pick[2]]-(test_list[test_pick[0]]*test_list[test_pick[2]])
        POspec = test_list[test_pick[1]]*test_list[test_pick[3]]
        POnpv = (POspec*(1-prev))/((1-POsens)*prev+POspec*(1-prev))
        if test_pick ==[0,1,0,1]:
            tests = 'Serology x Serology'
        elif test_pick ==[2,3,2,3]:
            tests = 'PCR x PCR'
        elif test_pick ==[0,1,2,3]:
            tests = 'Serology x PCR'
        elif test_pick ==[2,3,4,5]:
            tests = 'PCR x Vitals'
        elif test_pick ==[0,1,4,5]:
            tests = 'Serology x Vitals'
        else:
            print ('none')
        dt_list = [str(prev*100)+'%',tests,f'{POsens:2f}',f'{POspec:.3f}',f'{POnpv:.3f}',pop,pop*prev,f'{pop*prev*(1-POsens):.3f}']
        dt_df.loc[len(dt_df)]=dt_list

    npvSero = (test_list[1]*(1-prev))/((1-test_list[0])*prev+test_list[1]*(1-prev))
    npvPCR = (test_list[3]*(1-prev))/((1-test_list[2])*prev+test_list[3]*(1-prev))
    npvVit = (test_list[5]*(1-prev))/((1-test_list[4])*prev+test_list[5]*(1-prev))
    
    #build lists for dataframe
    prevalance = [str(prev*100)+'%']*3
    pop_size = [pop]*3
    FalseNegs = [pop*prev*(1-test_list[0]),pop*prev*(1-test_list[2]),pop*prev*(1-test_list[4])]
    infected = [pop*prev]*3
    sensitivities = [f'{test_list[0]:.3f}',f'{test_list[2]:.3f}',f'{test_list[4]:.3f}']
    specificities = [f'{test_list[1]:.3f}',f'{test_list[3]:.3f}',f'{test_list[5]:.3f}']
    negPredVals = [f'{npvSero:.3f}',f'{npvPCR:.3f}',f'{npvVit:.3f}']
    names = ['Sero','PCR','Vitals']
    data_tuples = list(zip(prevalance,names,sensitivities,specificities,negPredVals,pop_size,infected,FalseNegs))
    Test_Com = pd.DataFrame(data_tuples, columns=('Prevalence','Test Name','Sensitivity','Specificitiy','NegPredVal','Pop_Size','Infected','False_Negatives'))
    Test_Char = Test_Char.append(Test_Com)
Test_Char = Test_Char.append(dt_df)
Test_Char.reset_index(drop = True,inplace = True)
Test_Char.to_csv('FalseNegPrediction')

In [18]:
#Test_Char

In [19]:
#Test_Char
#Test_Char.iloc[[0,1,2,12,13,11,6,7,8,22,23,21] ]
#Test_Char.iloc[[0,6,1,7,2,8,12,22,13,23,11,21] ]




# 1st, 4th, 7th, 25th row + 1st 6th 7th columns


In [15]:
p1 = ['4.5% - 20%']*6
n1 = ['Serology','PCR','Vitals','PCR x Vitals','Serology x Vitals','PCR x Serology']
i1 = ['16-70']*6
fn1 = ['2.5-11.5','5.8 - 25.9','9.5 - 42','3.5 - 15.5','1.5 - 15.75','0.96 - 4.42']
cn1 = ['Prevalence','Test Name', '# Infected/350','False Negatives']

data_tuples = list(zip(p1,n1,i1,fn1))

In [20]:
df1 = pd.DataFrame(data_tuples, columns=cn1)


Test_Char.iloc[[0,1,2,12,13,11,6,7,8,22,23,21] ]
df1

,Prevalence,Test Name,Sensitivity,Specificitiy,NegPredVal,Pop_Size,Infected,False_Negatives
0,4.5%,Sero,0.836,0.977,0.992,1000,45.0,7.38
1,4.5%,PCR,0.630,0.990,0.983,1000,45.0,16.65
2,4.5%,Vitals,0.400,0.670,0.960,1000,45.0,27
12,4.5%,PCR x Vitals,0.778000,0.663,0.984,1000,45.0,9.990
13,4.5%,Serology x Vitals,0.901600,0.655,0.993,1000,45.0,4.428
11,4.5%,Serology x PCR,0.939320,0.967,0.997,1000,45.0,2.731
6,20.0%,Sero,0.836,0.977,0.960,1000,200.0,32.8
7,20.0%,PCR,0.630,0.990,0.915,1000,200.0,74
8,20.0%,Vitals,0.400,0.670,0.817,1000,200.0,120
22,20.0%,PCR x Vitals,0.778000,0.663,0.923,1000,200.0,44.400


,Prevalence,Test Name,# Infected/350,False Negatives
0,4.5% - 20%,Serology,16-70,2.5-11.5
1,4.5% - 20%,PCR,16-70,5.8 - 25.9
2,4.5% - 20%,Vitals,16-70,9.5 - 42
3,4.5% - 20%,PCR x Vitals,16-70,3.5 - 15.5
4,4.5% - 20%,Serology x Vitals,16-70,1.5 - 15.75
5,4.5% - 20%,PCR x Serology,16-70,0.96 - 4.42



## Combining Tests
### Parallel - done at same time, interp together
#### OR test
##### yields a positive diagnosis if either test is positive and a negative diagnosis if both tests are negative

Sens = SEa + SEb - (SEa × SEb)

Spec = SPa × SPb

#### AND test
##### yields a positive diagnosis only if both tests are positive and a negative diagnosis if either test is negative


Sens = SEa × SEb

Spec = SPa + SPb - (SPa × SPb)

### Series - done one at a time, 1st determines if 2nd done
#### OR test
##### if the first test is positive, the diagnosis is positive; otherwise, the second test is performed. If the second test is positive after a negative first test, then the diagnosis also is positive; otherwise, the diagnosis is negative


Sens = SEa + (1 - SEa) × SEb

Spec = SPa × SPb

#### AND test
##### if the first test is positive, the second test is performed. If the second test is positive, the diagnosis is positive; otherwise, the diagnosis is negative

Sens = SEa × SEb

Spec = SPa + (1 - SPa) × SPb

## Negative Predictive Value
#### What is the likelihood that this patient does not have the disease when the test result is negative?
The probability that the disease is absent given the pretest probability (prevalance) and negative test result

NPV = (Spec × (1-prev)) / (Spec × (1-prev) + (1-Sens) × prev




Clinical Evaluation of Diagnostic Tests
Susan Weinstein, Nancy A. Obuchowski and Michael L. Lieber
American Journal of Roentgenology. 2005;184: 14-19. 10.2214/ajr.184.1.01840014

Read More: https://www.ajronline.org/doi/abs/10.2214/ajr.184.1.01840014

### Assumptions
population = 350

prevalance = Layton, Salt Lake, and average % of UT tests coming back positive over the past week

### Question 1
If I use a single diagnostic test on my population of 350 people how many negative tests are likely false
- Test B has highest single test NPV

In Layton a person with a negative test has a 98-99% chance of being a true negative
- prevalance is 0.00052
- 0.1 people in my population are infected
- number of False Negatives = 0.003
    
In Salt Lake a person with a negative test has a 80-89% chance of being a true negative 
- prevalance is 0.00399
- 1.5 people in my population are infected
- number of False Negatives = 0.15
    
In All of UT a person with a negative test has a 97-99% chance of being a true negative
- prevalance is 0.045
- 16.1 people in my population are infected
- number of False Negatives = 3.2

### Question 2
If I use two tests in parallel on my population of 350 people how many negative tests are likely false
- Parallel OR has highest test NPV

In Layton a person with a negative test has a 98-99% chance of being a true negative
- prevalance is 0.00052
- 0.2 people in my population are infected
- number of False Negatives = 0.007
    
In Salt Lake a person with a negative test has a 80-89% chance of being a true negative 
- prevalance is 0.00399
- 1.4 people in my population are infected
- number of False Negatives = 0.059
    
In All of UT a person with a negative test has a 97-99% chance of being a true negative
- prevalance is 0.045
- 15.7 people in my population are infected
- number of False Negatives = 0.55

